In [1]:
import random 
from sage.all import Graph, RR, infinity, graphs        
import warnings
warnings.filterwarnings("ignore", message=".*Signature.*numpy.longdouble.*")

# OSNOVNE FUNKCIJE

In [2]:
# definiramo funkcijo za Somborjev indeks

def general_sombor_index(G, alpha):
    alpha = RR(alpha)
    deg = G.degree()  
    val = RR(0)        
    for u, v in G.edges(labels=False):
        du = deg[u]
        dv = deg[v]
        val += (du**2 + dv**2)**alpha
    return RR(val)


# definirajmo funkcijo za največjo stopnjo - delto

def max_degree(G):
    return max(G.degree(v) for v in G.vertices()) if G.num_verts() > 0 else 0

# MAJHNI GRAFI (n≤10)

## Funkcije za sistematično iskanje

In [3]:
# Funkcija generira vsa neizomorfna drevesa z n vozlišči, vsako drevo razvrsti glede na njegovo maksimalno stopnjo (Δ) ter vrne slovar, 
# kjer je ključ Δ, vrednost pa seznam dreves z isto maksimalno stopnjo.

def classify_trees_by_max_degree(n):
    """
    Vrne slovar:
        Δ -> [seznam neizomorfnih dreves z n vozlišči in maksimalno stopnjo Δ]
    """
    result = {}

    for T in graphs.trees(n):
        # izračun maximalne stopnje
        delta = max(T.degree(v) for v in T.vertices()) if T.num_verts() > 0 else 0

        if delta not in result:
            result[delta] = []

        result[delta].append(T)

    return result

In [4]:
# Funkcija sistematično pregleda vsa drevesa na n vozliščih (razvrščena po maksimalni stopnji) in za dano vrednost parametra alpha 
# izračuna njihov splošni Somborjev indeks. Vrne največjo doseženo vrednost indeksa ter seznam vseh dreves, ki to vrednost dosežejo.
# Če je return_all_best=True, vrne vsa optimalna drevesa, sicer le prvo najdeno.
# Za vsako optimalno drevo shrani njegov seznam povezav in maksimalno stopnjo.

def systematic_best_trees_all(n, alpha, return_all_best=True):
    best_val = -infinity
    best_list = []   # tukaj bomo shranili: (edges, delta)

    trees_by_delta = classify_trees_by_max_degree(n)

    for delta in trees_by_delta:
        for T in trees_by_delta[delta]:

            val = general_sombor_index(T, alpha)
            edges = T.edges(labels=False)
            maxdeg = max(T.degree(v) for v in T.vertices())

            # Nova najboljša vrednost
            if val > best_val + 1e-12:
                best_val = val
                best_list = [(edges, maxdeg)]

            # Če je enako najboljši in želimo vse najboljše
            elif abs(val - best_val) <= 1e-12 and return_all_best:
                best_list.append((edges, maxdeg))

    return best_val, best_list

In [5]:
# Funkcija, ki za vsak delta zračuna pri katerem drevesu je najboljši Somborjev indeks pri danem alpha.

def best_sombor_by_delta(n, alpha):
    """
    Za vsak Δ izračuna največji Somborjev indeks in vrne
    najboljša drevesa ter njihove Δ in robove.

    Vrne slovar:
    {
        Δ : {
                "best_val": vrednost Sombor indeksa,
                "best_trees": [ (edges, delta), ... ]
            }
    }
    """
    result = {}
    trees_by_delta = classify_trees_by_max_degree(n)

    for delta, tree_list in trees_by_delta.items():

        best_val = -infinity
        best_trees = []

        for T in tree_list:

            val = general_sombor_index(T, alpha)
            edges = T.edges(labels=False)
            maxdeg = max(T.degree(v) for v in T.vertices())

            # Nova najboljša vrednost
            if val > best_val + 1e-12:
                best_val = val
                best_trees = [(edges, maxdeg)]

            elif abs(val - best_val) <= 1e-12:
                best_trees.append((edges, maxdeg))

        result[delta] = {
            "best_val": best_val,
            "best_trees": best_trees
        }

    return result

## Preverjanje rezultatov (majhni grafi)

### Uporabljene alfe

In [6]:
alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]

In [7]:
# Napišemo funkcijo za sistematično preverjanje, da ni treba vsakič vsega znova pisati

def systematic_search(n, alpha_values):
    """
    Izvede sistematično in na maksimalni stopnji (delta) iskanje največjega
    Somborjevega indeksa za fiksno število vozlišč n in več različnih vrednosti
    parametra alpha.
    """

    for alpha in alpha_values:
        alpha_R = RR(alpha)

        print("=" * 60)
        print(f"α = {float(alpha_R):.2f}")
        print("=" * 60)

        # izračunamo največji Somborjev indeks za vsak Δ
        res = best_sombor_by_delta(n, alpha_R)

        for delta in sorted(res.keys()):
            print(f"Δ = {delta}")
            print("  največji Sombor indeks =", res[delta]["best_val"])
            for edges, d in res[delta]["best_trees"]:
                print("  edges:", edges)
            print()


        best_global_val, best_global_list = systematic_best_trees_all(n, alpha_R)

        max_delta = best_global_list[0][1]  # (edges, delta)

        print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha_R):.2f}")
        print(f"   Δ = {max_delta},  Sombor indeks = {best_global_val}")

        for edges, d in best_global_list:
            print("   najboljše drevo, edges:", edges)

        print("\n")


### n=2

In [8]:
systematic_search(2, alpha_values)

α = 0.05
Δ = 1
  največji Sombor indeks = 1.03526492384138
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 1,  Sombor indeks = 1.03526492384138
   najboljše drevo, edges: [(0, 1)]


α = 0.45
Δ = 1
  največji Sombor indeks = 1.36604025675440
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 1,  Sombor indeks = 1.36604025675440
   najboljše drevo, edges: [(0, 1)]


α = 0.50
Δ = 1
  največji Sombor indeks = 1.41421356237310
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 1,  Sombor indeks = 1.41421356237310
   najboljše drevo, edges: [(0, 1)]


α = 0.55
Δ = 1
  največji Sombor indeks = 1.46408569594563
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.55
   Δ = 1,  Sombor indeks = 1.46408569594563
   najboljše drevo, edges: [(0, 1)]


α = 0.95
Δ = 1
  največji Sombor indeks = 1.93187265784969
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.95
   Δ = 1,  Sombor indeks = 1.93187265784969
   najboljše drevo, edges: [(0, 1)]




### n=3

In [9]:
systematic_search(3, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 2.16759677346874
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 2,  Sombor indeks = 2.16759677346874
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.45
Δ = 2
  največji Sombor indeks = 4.12635413536159
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 2,  Sombor indeks = 4.12635413536159
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.50
Δ = 2
  največji Sombor indeks = 4.47213595499958
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 2,  Sombor indeks = 4.47213595499958
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.55
Δ = 2
  največji Sombor indeks = 4.84689373328531
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.55
   Δ = 2,  Sombor indeks = 4.84689373328531
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.95
Δ = 2
  največji Sombor indeks = 9.22680834590588
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.95
   Δ = 2,  Sombor indeks = 9.22680834590588
 

### n=4

In [10]:
systematic_search(4, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 3.27716624553658
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 3.36605536290589
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 3,  Sombor indeks = 3.36605536290589
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.45
Δ = 2
  največji Sombor indeks = 6.67547539000011
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 8.45514879379336
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 3,  Sombor indeks = 8.45514879379336
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.50
Δ = 2
  največji Sombor indeks = 7.30056307974577
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 9.48683298050514
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 3,  Sombor indeks = 9.48683298050514
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.55
Δ = 2
  največji Sombor indeks = 7.98523012487231
  edges: [(

### n=5

In [11]:
systematic_search(5, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 4.38673571760443
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 4.46466959319389
  edges: [(0, 1), (0, 3), (0, 4), (1, 2)]

Δ = 4
  največji Sombor indeks = 4.60874243131641
  edges: [(0, 1), (0, 2), (0, 3), (0, 4)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 4,  Sombor indeks = 4.60874243131641
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4)]


α = 0.45
Δ = 2
  največji Sombor indeks = 9.22459664463864
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 10.8715144444270
  edges: [(0, 1), (0, 3), (0, 4), (1, 2)]

Δ = 4
  največji Sombor indeks = 14.3140327308418
  edges: [(0, 1), (0, 2), (0, 3), (0, 4)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 4,  Sombor indeks = 14.3140327308418
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4)]


α = 0.50
Δ = 2
  največji Sombor indeks = 10.1289902044920
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 12.16617457

### n=6

In [12]:
systematic_search(6, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 5.49630518967227
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5)]

Δ = 3
  največji Sombor indeks = 5.64355698997166
  edges: [(0, 1), (0, 4), (0, 5), (1, 2), (1, 3)]

Δ = 4
  največji Sombor indeks = 5.70194155986322
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (1, 2)]

Δ = 5
  največji Sombor indeks = 5.88462336394871
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 5,  Sombor indeks = 5.88462336394871
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]


α = 0.45
Δ = 2
  največji Sombor indeks = 11.7737178992772
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5)]

Δ = 3
  največji Sombor indeks = 14.9452776986243
  edges: [(0, 1), (0, 4), (0, 5), (1, 2), (1, 3)]

Δ = 4
  največji Sombor indeks = 16.6487261588689
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (1, 2)]

Δ = 5
  največji Sombor indeks = 21.6624718279813
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

→ GLOBALNI MAKSIMUM pri α =

### n=7

In [13]:
systematic_search(7, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 6.60587466174012
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5), (5, 6)]

Δ = 3
  največji Sombor indeks = 6.76174241291905
  edges: [(0, 1), (0, 4), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 6.87521267517925
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (1, 2), (1, 3)]

Δ = 5
  največji Sombor indeks = 6.97486529247889
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (1, 2)]

Δ = 6
  največji Sombor indeks = 7.18722658362132
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 6,  Sombor indeks = 7.18722658362132
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]


α = 0.45
Δ = 2
  največji Sombor indeks = 14.3228391539157
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5), (5, 6)]

Δ = 3
  največji Sombor indeks = 17.6166747534923
  edges: [(0, 1), (0, 4), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 20.6289900232642
  edges: [(0, 

### n=8

In [14]:
systematic_search(8, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 7.71544413380796
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7)]

Δ = 3
  največji Sombor indeks = 7.92105861703743
  edges: [(0, 1), (0, 4), (0, 7), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 8.10232076197734
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4)]

Δ = 5
  največji Sombor indeks = 8.14455294530322
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3)]

Δ = 6
  največji Sombor indeks = 8.27570347688230
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2)]

Δ = 7
  največji Sombor indeks = 8.51229253461060
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 7,  Sombor indeks = 8.51229253461060
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]


α = 0.45
Δ = 2
  največji Sombor indeks = 16.8719604085542
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7)]



### n=9

In [15]:
systematic_search(9, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 8.82501360587581
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7), (7, 8)]

Δ = 3
  največji Sombor indeks = 9.03924403998482
  edges: [(0, 1), (0, 5), (0, 8), (1, 2), (2, 3), (2, 4), (5, 6), (5, 7)]

Δ = 4
  največji Sombor indeks = 9.23628634625770
  edges: [(0, 1), (0, 5), (1, 2), (1, 3), (1, 4), (5, 6), (5, 7), (5, 8)]

Δ = 5
  največji Sombor indeks = 9.36829073992565
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2), (1, 3), (1, 4)]

Δ = 6
  največji Sombor indeks = 9.44304489036981
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2), (1, 3)]

Δ = 7
  največji Sombor indeks = 9.59963895871060
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2)]

Δ = 8
  največji Sombor indeks = 9.85679342400157
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 8,  Sombor indeks = 9.85679342400157
   najboljše drevo, edges: [(0, 1), (0,

### n=10

In [16]:
systematic_search(10, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 9.93458307794365
  edges: [(0, 1), (0, 6), (1, 2), (2, 3), (3, 4), (4, 5), (6, 7), (7, 8), (8, 9)]

Δ = 3
  največji Sombor indeks = 10.1985602441032
  edges: [(0, 1), (0, 6), (0, 9), (1, 2), (1, 5), (2, 3), (2, 4), (6, 7), (6, 8)]
  edges: [(0, 1), (0, 4), (0, 7), (1, 2), (1, 3), (4, 5), (4, 6), (7, 8), (7, 9)]

Δ = 4
  največji Sombor indeks = 10.3843699874526
  edges: [(0, 1), (0, 5), (0, 9), (1, 2), (1, 3), (1, 4), (5, 6), (5, 7), (5, 8)]

Δ = 5
  največji Sombor indeks = 10.6314391729766
  edges: [(0, 1), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5)]

Δ = 6
  največji Sombor indeks = 10.6643411415811
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4)]

Δ = 7
  največji Sombor indeks = 10.7653872571051
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3)]

Δ = 8
  največji Sombor indeks = 10.9433745911232
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0,

# VELIKI GRAFI (n>10)

## Funkcije za stohastično iskanje

In [17]:
# Lokalni premik (za stohastično iskanje) - premik lista

def random_tree_move(G, Delta, max_attempts=50):
    """
    Izberemo naključni list l z njegovim sosedom p, list l odklopimo 
    od p, in ga priklopimo na drugo vozlišče q s stopnjo < Delta. Ta 
    premik ohrani graf drevo in ne poveča maksimalne stopnje.
    """

    H = G.copy()

    for _ in range(max_attempts):
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        if not leaves:
            return G

        leaf = random.choice(leaves)
        parent = H.neighbors(leaf)[0]

        candidates = [
            v for v in H.vertices()
            if v != leaf and v != parent and H.degree(v) < Delta
        ]

        if not candidates:
            continue

        new_parent = random.choice(candidates)

        H.delete_edge(leaf, parent)
        H.add_edge(leaf, new_parent)

        if H.is_tree():
            return H

        H.delete_edge(leaf, new_parent)
        H.add_edge(leaf, parent)

    return G

In [18]:
# Funkcija generira naključno drevo na n vozliščih, katerega maksimalna stopnja
# je natanko Delta. Drevo je zgrajeno tako, da najprej zagotovi vozlišče
# s stopnjo Delta, nato pa preostala vozlišča postopoma dodaja brez kršitve
# omejitve maksimalne stopnje.


def random_tree_with_exact_max_degree(n, Delta):
    """
    Zahteva n >= Delta + 1 in Delta >= 2.
    """

    G = Graph()
    G.add_vertices(range(n))

    deg = {v: 0 for v in range(n)}

    # naredimo zvezdo da zagotovimo max degree = Delta
    center = 0
    for leaf in range(1, Delta + 1):
        G.add_edge(center, leaf)
        deg[center] += 1
        deg[leaf] += 1

    next_vertex = Delta + 1

    # dodamo preostala vozlišča
    while next_vertex < n:
        # kandidati so vozlišča, ki so že ustvarjena
        candidates = [u for u in range(next_vertex) if deg[u] < Delta]

        parent = random.choice(candidates)
        child = next_vertex

        G.add_edge(parent, child)
        deg[parent] += 1
        deg[child] += 1

        next_vertex += 1

    return G

In [19]:
# Funkcija izvede stohastično iskanje tipa hill-climbing za drevesa na n vozliščih,
# katerih maksimalna stopnja je natanko Delta, z namenom maksimizacije
# splošnega Somborjevega indeksa. Iskanje uporablja več naključnih
# začetkov in lokalne premike, končni rezultat pa je najboljše najdeno drevo
# in pripadajoča vrednost indeksa.

def stochastic_search_fixed_n(n, Delta, alpha,
                              n_starts=None,
                              n_steps=None,
                              start_tree=None):
    """
    Parametri:
    n_starts : int ali None
        Število naključnih ponovnih zagonov (privzeto: min(n//2, 30)).
    n_steps : int ali None
        Število lokalnih premikov na posamezen zagon
        (privzeto: min(35*n, 6000)).
    """

    if n_starts is None:
        n_starts = min(n // 2, 30)

    if n_steps is None:
        n_steps = min(35 * n, 6000)

    alpha = RR(alpha)

    best_val = -infinity
    best_tree = None

    for _ in range(n_starts):

        # začnemo z drevesom, ki ima Δ(T)=Δ
        G = random_tree_with_exact_max_degree(n, Delta)
        val = general_sombor_index(G, alpha)

        for _ in range(n_steps):

            H = random_tree_move(G, Delta)
            if H is G:
                continue

            if max_degree(H) != Delta:
                continue

            new_val = general_sombor_index(H, alpha)

            # hill-climbing 
            if new_val > val + 1e-12:
                G, val = H, new_val

        # posodobi najboljše globalno drevo
        if val > best_val + 1e-12:
            best_val = val
            best_tree = G.copy()

    return best_val, best_tree


## Preverjanje rezultatov (veliki grafi)

In [20]:
# To dodamo zato, da naslednja funkcija dela hitreje. 
# Vrne n_starts in n_steps glede na n in Delta.

def adaptive_params(n, Delta):
    if Delta <= 3:
        return 3, 600
    elif Delta <= 6:
        return 5, 1200
    elif Delta <= 10:
        return 7, 2000
    elif Delta <= n//2:
        return 9, 3000
    else:
        return 6, 2000


In [21]:
# Funkcija za dano število vozlišč n (privzamemo n > 10) izvede stohastično
# iskanje dreves, ki maksimizirajo splošni Somborjev indeks, za več izbranih
# vrednosti parametra alpha in maksimalne stopnje Delta. Za vsako kombinacijo
# izpiše osnovne strukturne lastnosti optimalnega drevesa ter njegov seznam povezav.

def scan_alpha_and_delta(
    n,
    alpha_values=None,
    Delta_values=None,
    n_starts=None,
    n_steps=None,
    print_edges=True):
    
    """
    Stohastično iskanje za fiksno število vozlišč n.
    Če seznama alpha_values ali Delta_values nista podana, se uporabijo
    smiselne privzete vrednosti.
    """

    # privzete vrednosti
    if alpha_values is None:
        alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]

    if Delta_values is None:
        Delta_values = [
            2,          # pot
            3, 4, 5,    # prvo razvejanje
            10,         # srednja stopnja
            n // 4,     # prehod
            n // 2,     # visoka koncentracija
            n - 1       # zvezda
        ]

    # vedno sortiramo delta
    Delta_values = sorted(set(Delta_values))

    for alpha in alpha_values:
        alpha_R = RR(alpha)

        print("=" * 60)
        print(f"α = {float(alpha_R):.2f}")
        print("=" * 60)

        prev_tree = None   # seed reset za vsak alpha

        for Delta in Delta_values:
            print(f"Δ = {Delta}")

            # adaptivni parametri (funkcija, ki smo jo definirali prej)
            if n_starts is None or n_steps is None:
                n_s, n_p = adaptive_params(n, Delta)
            else:
                n_s, n_p = n_starts, n_steps

            best_val, best_tree = stochastic_search_fixed_n(
                n=n,
                Delta=Delta,
                alpha=alpha_R,
                n_starts=n_s,
                n_steps=n_p,
                start_tree=prev_tree
            )

            assert best_tree.is_tree()
            assert max_degree(best_tree) == Delta

            deg_seq = sorted(best_tree.degree())

            # števec vozlišč z maksimalno stopnjo
            count_max = deg_seq.count(Delta)

            # druga največja stopnja
            unique_degs = sorted(set(deg_seq))
            second_max = unique_degs[-2] if len(unique_degs) >= 2 else None

            print(f"  best value           = {best_val}")
            print(f"  degree seq           = {deg_seq}")
            print(f"  #vertices with deg Δ = {count_max}")
            if second_max is not None:
                print(f"  second max degree    = {second_max}")

            if print_edges:
                print(f"  edges                = {best_tree.edges(labels=False)}")

            print()

            prev_tree = best_tree   # seed za naslednji delta

        print()


### Testiranje za n = 60

In [22]:
scan_alpha_and_delta(n=60)


α = 0.05
Δ = 2
  best value           = 65.4130566813360
  degree seq           = [1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
  #vertices with deg Δ = 58
  second max degree    = 1
  edges                = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 8), (4, 5), (5, 6), (6, 7), (7, 9), (8, 15), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 16), (15, 21), (16, 17), (17, 18), (18, 19), (19, 20), (20, 28), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 30), (28, 29), (29, 32), (30, 31), (31, 36), (32, 33), (33, 34), (34, 35), (35, 37), (36, 38), (37, 40), (38, 39), (39, 42), (40, 41), (41, 43), (42, 44), (43, 45), (44, 52), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 53), (52, 58), (53, 54), (54, 55), (55, 56), (56, 57), (58, 59)]

Δ = 3
  best value           = 67.0127085772345
  degree seq           = [1, 1, 1, 1,

In [23]:
#scan_alpha_and_delta(n=123)